In [2]:
import numpy as np
data = np.load('../data/tensor.npy')

from skimage.util.shape import view_as_windows as viewW
from sklearn.model_selection import train_test_split

import keras.backend as K
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Dense, Masking
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.losses import *

Using TensorFlow backend.


In [177]:
# #if on AWS:
# import numpy as np
# from s3fs.core import S3FileSystem
# s3 = S3FileSystem()

# key = 'tensor.npy'
# bucket = 'sagemaker-wavey-playlists'

# data = np.load(s3.open('{}/data/{}'.format(bucket, key)))

In [3]:
def independent_shift(A,r):
    '''
    A - TENSOR
    r - array of shifts
    '''
    rows, column_indices = np.ogrid[:A.shape[0], :A.shape[1]]

    r[r < 0] += A.shape[1]
    column_indices = column_indices - r[:, np.newaxis]

    return A[rows, column_indices]

In [4]:
data = independent_shift(data,np.isnan(data)[:,:,0].sum(axis=1)-50)

In [5]:
data.shape

(72051, 50, 9)

## Train Test Lit 🔥

In [7]:
mask = 0

X = np.where(np.isnan(data), mask, data)[:,:-1,:]
Y = np.where(np.isnan(data), mask, data)[:,1:,1:] 

x_train,x_test,y_train,y_test = train_test_split(X,Y,train_size=0.9,random_state=3000) 

## Custom Loss Functions

In [182]:
def masked_mse(mask):
    def loss(y_true,y_pred):
        if y_true == np.array([mask for mask in range(9)]):
            return 0
        else:
            return mean_squared_error(y_true,y_pred)
    return loss

def masked_mae(mask):
    def loss(y_true,y_pred):
        if y_true == np.array([mask for mask in range(9)]):
            return 0
        else:
            return mean_absolute_error(y_true,y_pred)
    return loss

def masked_logcosh(mask):
    def loss(y_true,y_pred):
        return logcosh(y_true,y_pred)*(y_true != np.array([mask for mask in range(9)]))
    return loss

def gaussian(x):
    return K.exp(-K.pow(x,2))

## Define and Compile Keras Model

In [16]:
model = Sequential()
model.add(Masking(mask_value= mask, input_shape = (None,9)))
model.add(SimpleRNN(
    16, 
    activation='linear', 
    return_sequences=True, 
    kernel_initializer='random_uniform',
))
model.add(SimpleRNN(
    16, 
    activation='linear',
    return_sequences=True,
    kernel_initializer='random_uniform',
))
model.add(Dense(8, activation='linear', kernel_initializer='random_uniform',))

In [17]:
model.compile(loss='mae', optimizer='adam') #optimizer=RMSprop(lr=0.005),)#metrics=['mae'])

In [18]:
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

Train on 64845 samples, validate on 7206 samples
Epoch 1/10
64845/64845 [==============================] - 51s 794us/step - loss: 0.6261 - val_loss: 0.5988
Epoch 2/10
64845/64845 [==============================] - 44s 674us/step - loss: 0.5975 - val_loss: 0.5965
Epoch 3/10
64845/64845 [==============================] - 41s 632us/step - loss: 0.5923 - val_loss: 0.5899
Epoch 4/10
64845/64845 [==============================] - 40s 618us/step - loss: 0.5901 - val_loss: 0.5899
Epoch 5/10
64845/64845 [==============================] - 39s 608us/step - loss: 0.5900 - val_loss: 0.5897
Epoch 6/10
64845/64845 [==============================] - 38s 581us/step - loss: 0.5899 - val_loss: 0.5897
Epoch 7/10
64845/64845 [==============================] - 37s 578us/step - loss: 0.5892 - val_loss: 0.5882
Epoch 8/10
64845/64845 [==============================] - 39s 603us/step - loss: 0.5872 - val_loss: 0.5864
Epoch 9/10
64845/64845 [==============================] - 44s 675us/step - loss: 0.5868 - val_l

In [19]:
#mae_optimized_model_rmsprop = model
mae_optimized_model_adam = model

In [20]:
mae_optimized_model_adam.save("model.h5")

#### Best Models
- logcosh 0.2376 / mae 0.5930 - linear/linear - logcosh loss - RMSProp Optimizer
- mae 0.5898 - linear/linear - mae loss - RMSProp Optimizer
- mae 0.5845 - linear/linear - mae loss - Adam Optimizer


In [203]:
def predict_sample(sample,model):
    print('input')
    print(sample) 
    print('\n','output')
    print(model.predict(np.array([sample]))[0,-1]) 

In [206]:
predict_sample(X[2,45:],mae_optimized_model_adam) 

input
[[ 1.          0.88604374 -0.05037592 -1.08699789 -0.73480564 -0.49739257
   0.14999728 -1.05524551 -0.94557616]
 [ 1.         -0.88546059 -0.50476689  0.32647887 -0.73490683 -0.49739257
   1.39194778 -0.81868234 -0.15756452]
 [ 1.         -0.60661889  0.1367403   0.63672234 -0.73490683 -1.16714521
   1.56333783 -0.96747411 -0.50418656]
 [ 1.         -0.9506063   0.3089365   1.35603468 -0.73490683 -0.40439023
   0.73495451 -0.19753763  0.15131257]]

 output
[-0.72644746  0.06494095  0.60666335 -0.7427842  -0.40725166  0.90282977
 -0.61902374 -0.09285215]
